In [6]:
import pickle
import spacy
import os
import re, string, unicodedata
import nltk
!pip install contractions
!pip install inflect
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
def DoctoDict():
  file_Names = os.listdir("/content/drive/MyDrive/IR_ASSIGNMENT_1/stories1")
  file_Paths = []
  for i in range(len(file_Names)):
    file_Paths.append("/content/drive/MyDrive/IR_ASSIGNMENT_1/stories1/"+file_Names[i])
  corpus = {}
  for i in range(len(file_Paths)):
    with open(file_Paths[i],encoding = "latin-1") as f_input:
      corpus[file_Names[i]] = [f_input.read()]
  return corpus

In [8]:
CORPUS = DoctoDict()
with open('CORPUS.pickle', 'wb') as handle:
  pickle.dump(CORPUS, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
with open('/content/CORPUS.pickle', 'rb') as handle:
  corpus = pickle.load(handle)

len(corpus.keys())

467

In [10]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    text=re.sub('\n',' ',text)
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def get_wordnet_pos(tag):
    tag_dict = {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

# def lemmatize(words):
#     """Lemmatize words in list of tokenized words"""
#     lemmatizer = WordNetLemmatizer()
#     lemmas = []
#     posTagged = nltk.pos_tag(words)
#     wordnetTagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1][0])), posTagged))
#     for word,tag in wordnetTagged:
#       lemma = lemmatizer.lemmatize(word,tag)
#       lemmas.append(lemma)
#     return lemmas

def lemmatizeSpacy(words):
  sent = ""
  lwords = []
  for word in words:
    sent += word + " " 
  model = spacy.load("en_core_web_sm")
  tokens = model(sent)
  for i in range(len(tokens)):
    lwords.append((tokens[i].lemma_,i+1))
  # for token in tokens:
  #   lwords.append(token.lemma_)
  return lwords

def preProcess_html(fileName):
    sample = denoise_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = lemmatizeSpacy(words)
    words = remove_stopwords(words)
    # words = lemmatize(words)
    # words = lemmatizeSpacy(words)
    return words  

def clean_text(text):
    # text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z0-9A-Z]',' ',text)
    text=re.sub(' +',' ',text)
    return text

def preProcessotherfiles(fileName):
    sample = clean_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatizeSpacy(words)    
    return words 


In [11]:
for i in corpus.keys():
  if i.endswith(".html") or i.endswith(".header"):
    corpus[i][0] = preProcess_html(corpus[i][0])
  else:
    corpus[i][0] = preProcessotherfiles(corpus[i][0])

In [12]:
len(corpus.keys())

467

In [13]:
print(corpus['life.txt'])

[[('may', 1), ('well', 2), ('start', 3), ('write', 4), ('first', 5), ('writing', 6), ('sort', 7), ('mood', 8), ('secondly', 9), ('topic', 10), ('bother', 11), ('time', 12), ('long', 13), ('since', 14), ('put', 15), ('write', 16), ('sort', 17), ('text', 18), ('file', 19), ('essay', 20), ('subject', 21), ('tonight', 22), ('craving', 23), ('write', 24), ('gather', 25), ('idea', 26), ('settle', 27), ('brain', 28), ('proceed', 29), ('write', 30), ('really', 31), ('exist', 32), ('hell', 33), ('thought', 34), ('run', 35), ('dave', 36), ('mind', 37), ('sit', 38), ('wooden', 39), ('floor', 40), ('room', 41), ('late', 42), ('night', 43), ('exact', 44), ('time', 45), ('sure', 46), ('computer', 47), ('autodiale', 48), ('hack', 49), ('code', 50), ('end', 51), ('room', 52), ('small', 53), ('dias', 54), ('type', 55), ('niche', 56), ('lay', 57), ('keyboard', 58), ('setup', 59), ('various', 60), ('book', 61), ('magazine', 62), ('lie', 63), ('strew', 64), ('title', 65), ('within', 66), ('range', 67), ('

In [15]:
# with open('DocTerms_SpacyLemm.pickle', 'wb') as handle:
  # pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/DocTerms_SpacyLemm.pickle', 'rb') as handle:
  docT = pickle.load(handle)

In [16]:
docT.keys()

dict_keys(['tree.txt', 'consumdr.hum', 'snow.txt', 'contrad1.hum', 'candle.hum', 'spiders.txt', 'timem.hac', 'aluminum.hum', 'life.txt', 'sight.txt', 'cameloto.hum', 'beyond.hum', 'partya.txt', 'nitepeek.sto', 'blind.txt', 'altside.hum', 'abyss.txt', 't_zone.jok', 'fantas.hum', 'advsayed.txt', 'eyeargon.hum', 'elite.app', 'rocket.sf', 'corcor.hum', 'elveshoe.txt', 'ab40thv.txt', 'gold3ber.txt', 'game.txt', 'excerpt.txt', 'knuckle.txt', 'emperor3.txt', 'empnclot.txt', 'abbey.txt', 'advtthum.txt', 'archive', 'wolfcran.txt', 'immorti.hum', 'wlgirl.txt', 'cooldark.sto', 'wolf7kid.txt', 'vday.hum', 'imagin.hum', 'adv_alad.txt', 'wombat.und', 'gemdra.txt', 'aircon.txt', 'wolflamb.txt', 'testpilo.hum', 'valen', 'confilct.fun', 'narciss.txt', 'enchdup.hum', 'ladylust.hum', 'tcoa.txt', 'enginer.txt', 'encamp01.txt', 'korea.s', 'taxnovel.txt', 'greedog.txt', 'keepmodu.txt', 'vainsong.txt', 'goldgoos.txt', 'quickfix', 'goldbug.poe', 'omarsheh.txt', 'grav', 'empty.txt', 'oxfrog.txt', 'gulliver.txt